# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-c6e02a72bf-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the common aspects between movies directed by Woody Allen or Quentin Tarantino. We want to know the people that worked for both directors with some numerical analyses, what are the differences in terms of budget for their movies, who won more Academy Awards. 

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q2526255`| Director       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`    | Quentin Tarantino      | node |





Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**


The workload should:


1. Identify the BGP for films

2. Identify the BGP for directors

3. Identify the BGP for workers in a films

4. Compare the workers amongst the films directed by the two directors

5. Return some numerical comparison between the two directors (e.g., how many workers in Tarantino's movies who also worked in Allen's films?, what is the film with the highest number of shared actors? Who is the most used actor by both the directors? etc. )

6. Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

7. Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films not only personal awards).

    7.1 Find the BGP for Academy Awards 

    7.2 Find the related subproperties

    7.3 Find how they are related to the directors
    
    7.4 Are there alternative queries to get the same result?



In [1]:
# start your workflow here

In [2]:
queryString = """
SELECT *
WHERE { 

wd:Q25089 ?p ?obj .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1006'), ('obj', 'http://data.bibliotheken.nl/id/thes/p06917654X')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1015'), ('obj', 'https://livedata.bibsys.no/authority/90085344')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1150'), ('obj', 'http://rvk.uni-regensburg.de/nt/HU+3027')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1741'), ('obj', 'http://data.beeldengeluid.nl/gtaa/76199')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/59077912')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163'), ('obj', 'http://id.worldcat.org/fast/1897483')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('obj', 'https://d-nb.info/gnd/118502077')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.go

334

## Task 1

In [6]:
# get predicates from Woody Allen

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 

wd:Q25089 ?p ?obj .

?p <http://schema.org/name> ?plbl

FILTER (!CONTAINS (?plbl, "ID") && !CONTAINS (?plbl, "username") && !CONTAINS (?plbl, "code"))

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('plbl', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('plbl', 'Commons gallery')]
[('p', 'http://www.wikidata.org/prop/direct/P213'), ('plbl', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P5587'), ('plbl', 'Libris-URI')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('plbl', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('plbl', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P8625'), ('plbl', 'bibliography')]
[('p', 'http://www.wikidata.org/prop/direct/P1477'), ('plbl', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('plbl', 'child')]
[('p', 'http://www.wikidata.org/prop/direct/P7763'), ('plbl', 'copyright status as a creator')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('plbl', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('plbl', 'date of birth')]
[('p', 'http://www.wik

47

In [12]:
# get filmography of Woody Allen

queryString = """
SELECT DISTINCT ?o ?olbl ?o2 ?o2lbl
WHERE { 

wd:Q25089 wdt:P1283 ?o .

?o wdt:P31 ?o2 .

?o <http://schema.org/name> ?olbl .
?o2 <http://schema.org/name> ?o2lbl .

} 
ORDER BY ?olbl
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2455835'), ('olbl', 'Woody Allen filmography'), ('o2', 'http://www.wikidata.org/entity/Q1371849'), ('o2lbl', 'filmography')]


1

In [15]:
# get predicates from "Woody Allen filmography"

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 

wd:Q2455835 ?p ?o .

?p <http://schema.org/name> ?plbl .

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('plbl', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('plbl', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('plbl', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('plbl', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('plbl', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('plbl', 'main subject')]


6

In [14]:
# get object from "Woody Allen filmography"->"is a list of"

queryString = """
SELECT DISTINCT ?o ?olbl
WHERE { 


wd:Q2455835 wdt:P360 ?o .

?o <http://schema.org/name> ?olbl .

} 
ORDER BY ?olbl
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11424'), ('olbl', 'film')]


1

In [16]:
# get predicates from node "film"

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 


wd:Q11424 ?p ?o .

?p <http://schema.org/name> ?plbl .

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5198'), ('plbl', 'ASC Leiden Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('plbl', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7033'), ('plbl', 'Australian Educational Vocabulary ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6200'), ('plbl', 'BBC News topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1617'), ('plbl', 'BBC Things ID')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('plbl', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('plbl', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('plbl', 'Biblioteca Nacional de España ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('plbl', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7685'), ('plbl', 'CALS Encyclopedia of Arkansas ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('plbl', 'Commons category

75

In [17]:
# get class of which "film" is a subclass

queryString = """
SELECT DISTINCT ?o ?olbl
WHERE { 


wd:Q11424 wdt:P279 ?o .

?o <http://schema.org/name> ?olbl .

} 
ORDER BY ?olbl
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2431196'), ('olbl', 'audiovisual work')]
[('o', 'http://www.wikidata.org/entity/Q10301427'), ('olbl', 'moving image')]
[('o', 'http://www.wikidata.org/entity/Q20937557'), ('olbl', 'series')]
[('o', 'http://www.wikidata.org/entity/Q4502142'), ('olbl', 'visual artwork')]


4

In [19]:
# get predicates to "film"

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 


?s ?p wd:Q11424 .

?p <http://schema.org/name> ?plbl .

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1204'), ('plbl', "Wikimedia portal's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('plbl', 'category combines topics')]
[('p', 'http://www.wikidata.org/prop/direct/P4224'), ('plbl', 'category contains')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('plbl', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('plbl', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('plbl', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('plbl', 'distribution format')]
[('p', 'http://www.wikidata.org/prop/direct/P8115'), ('plbl', 'eligible recipient')]
[('p', 'http://www.wikidata.org/prop/direct/P1574'), ('plbl', 'exemplar of')]
[('p', 'http://www.wikidata.org/prop/direct/P2079'), ('plbl', 'fabrication method')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('plbl', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1074'), ('plbl', 'fictional 

37

In [22]:
# GOAL (1)
# get films

queryString = """
SELECT DISTINCT ?film ?flbl
WHERE { 


?film wdt:P31 wd:Q11424 .

?film <http://schema.org/name> ?flbl .

} 
ORDER BY ?flbl
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q4540204'), ('flbl', '!Women Art Revolution')]
[('film', 'http://www.wikidata.org/entity/Q4032288'), ('flbl', '"713" Requests Permission to Land')]
[('film', 'http://www.wikidata.org/entity/Q93765322'), ('flbl', '"By the railroad" film')]
[('film', 'http://www.wikidata.org/entity/Q615254'), ('flbl', '"Crocodile" Dundee')]
[('film', 'http://www.wikidata.org/entity/Q1140745'), ('flbl', '"Crocodile" Dundee II')]
[('film', 'http://www.wikidata.org/entity/Q16691116'), ('flbl', '"Dream" Agency')]
[('film', 'http://www.wikidata.org/entity/Q3596104'), ('flbl', '"FF.SS." – Cioè: "...che mi hai portato a fare sopra a Posillipo se non mi vuoi più bene?"')]
[('film', 'http://www.wikidata.org/entity/Q4540285'), ('flbl', '"Good Hair" and Other Dubious Distinctions')]
[('film', 'http://www.wikidata.org/entity/Q93764465'), ('flbl', '"Interrogation", film, 1982')]
[('film', 'http://www.wikidata.org/entity/Q821560'), ('flbl', '"Pimpernel" Smith')]


10

## Task 2

In [25]:
# get predicate between "Quentin Tarantino" and "Director"

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 


wd:Q3772 ?p wd:Q2526255 .

?p <http://schema.org/name> ?plbl .

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('plbl', 'occupation')]


1

In [27]:
# GOAL (2)
# get all directors

queryString = """
SELECT DISTINCT ?s ?slbl
WHERE { 


?s wdt:P106 wd:Q2526255 .

?s <http://schema.org/name> ?slbl .

} 
ORDER BY ?slbl
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4544812'), ('slbl', '(Jo)Style')]
[('s', 'http://www.wikidata.org/entity/Q106992033'), ('slbl', '44aen')]
[('s', 'http://www.wikidata.org/entity/Q30923004'), ('slbl', '@MARU_movie')]
[('s', 'http://www.wikidata.org/entity/Q5673777'), ('slbl', 'A Harsha')]
[('s', 'http://www.wikidata.org/entity/Q22276674'), ('slbl', 'A J Mintu')]
[('s', 'http://www.wikidata.org/entity/Q93443565'), ('slbl', 'A. B. Comathiere')]
[('s', 'http://www.wikidata.org/entity/Q4647568'), ('slbl', 'A. B. Raj')]
[('s', 'http://www.wikidata.org/entity/Q4647596'), ('slbl', 'A. Bhimsingh')]
[('s', 'http://www.wikidata.org/entity/Q2818908'), ('slbl', 'A. Blaine Miller')]
[('s', 'http://www.wikidata.org/entity/Q4647628'), ('slbl', 'A. C. Tirulokchandar')]
[('s', 'http://www.wikidata.org/entity/Q278612'), ('slbl', 'A. D. Flowers')]
[('s', 'http://www.wikidata.org/entity/Q4647692'), ('slbl', 'A. Dean Bell')]
[('s', 'http://www.wikidata.org/entity/Q278657'), ('slbl', 'A. Edwar

20

## Task 3

In [33]:
# get predicates to a film

queryString = """
SELECT DISTINCT ?p 
WHERE { 

?s ?p wd:Q3596104 .

?p <http://schema.org/name> ?plbl .

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [34]:
# get predicates from a film

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 

wd:Q3596104 ?p ?o .

?p <http://schema.org/name> ?plbl .

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4529'), ('plbl', 'Douban film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2704'), ('plbl', 'EIDR content ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('plbl', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P345'), ('plbl', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2603'), ('plbl', 'Kinopoisk film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6127'), ('plbl', 'Letterboxd film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4947'), ('plbl', 'TMDb movie ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('plbl', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P462'), ('plbl', 'color')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('plbl', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('plbl', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('plbl', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), 

20

In [40]:
# GOAL (3)
# get all workers and their roles for each film

queryString = """
SELECT DISTINCT ?film ?flbl ?role ?rlbl ?worker ?wlbl
WHERE { 

?film wdt:P31 wd:Q11424 ;
    ?role ?worker .
    
?worker wdt:P31 wd:Q5 .

?film <http://schema.org/name> ?flbl .
?worker <http://schema.org/name> ?wlbl .
?role <http://schema.org/name> ?rlbl .

} 
ORDER BY ?flbl ?wlbl
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q4540204'), ('flbl', '!Women Art Revolution'), ('role', 'http://www.wikidata.org/prop/direct/P86'), ('rlbl', 'composer'), ('worker', 'http://www.wikidata.org/entity/Q2940239'), ('wlbl', 'Carrie Brownstein')]
[('film', 'http://www.wikidata.org/entity/Q4540204'), ('flbl', '!Women Art Revolution'), ('role', 'http://www.wikidata.org/prop/direct/P57'), ('rlbl', 'director'), ('worker', 'http://www.wikidata.org/entity/Q6709060'), ('wlbl', 'Lynn Hershman Leeson')]
[('film', 'http://www.wikidata.org/entity/Q4540204'), ('flbl', '!Women Art Revolution'), ('role', 'http://www.wikidata.org/prop/direct/P58'), ('rlbl', 'screenwriter'), ('worker', 'http://www.wikidata.org/entity/Q6709060'), ('wlbl', 'Lynn Hershman Leeson')]
[('film', 'http://www.wikidata.org/entity/Q4032288'), ('flbl', '"713" Requests Permission to Land'), ('role', 'http://www.wikidata.org/prop/direct/P86'), ('rlbl', 'composer'), ('worker', 'http://www.wikidata.org/entity/Q4373271'), (

40

## Task 4

In [54]:
# get all films directed by Quentin Tarantino or Woody Allen

queryString = """
SELECT DISTINCT ?film ?flbl ?dir
WHERE { 

{
    ?film wdt:P31 wd:Q11424 ;
        ?role ?worker .

    ?film wdt:P57 wd:Q3772 .

    ?film <http://schema.org/name> ?flbl .
    wd:Q3772 <http://schema.org/name> ?dir .
}
UNION
{
    ?film wdt:P31 wd:Q11424 ;
        ?role ?worker .

    ?film wdt:P57 wd:Q25089 .

    ?film <http://schema.org/name> ?flbl .
    wd:Q25089 <http://schema.org/name> ?dir .
}

FILTER (?role != wdt:P57)

} 
ORDER BY ?flbl
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1306629'), ('flbl', "A Midsummer Night's Sex Comedy"), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/entity/Q42307699'), ('flbl', 'A Rainy Day in New York'), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('flbl', 'Alice'), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/entity/Q233464'), ('flbl', 'Annie Hall'), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/entity/Q541185'), ('flbl', 'Another Woman'), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/entity/Q614009'), ('flbl', 'Anything Else'), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/entity/Q806092'), ('flbl', 'Bananas'), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/entity/Q2907178'), ('flbl', 'Blue Jasmine'), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/entity/Q729026'), ('flbl', 'Broadway Danny Rose'), ('dir', 'Woody Allen')]
[('film', 'http://www.wikidata.org/en

40

In [58]:
# get objects of Woody Allen for "award received", "list of work" and "work period (start)"

queryString = """
SELECT DISTINCT ?aw ?awlbl ?low ?lowlbl ?wp
WHERE { 

wd:Q25089 wdt:P166 ?aw ;
    wdt:P1455 ?low ;
    wdt:P2031 ?wp .

?aw <http://schema.org/name> ?awlbl .
?low <http://schema.org/name> ?lowlbl .

} 
"""

print("Results")
run_query(queryString)

Results
[('aw', 'http://www.wikidata.org/entity/Q1415017'), ('awlbl', 'Writers Guild of America Award'), ('low', 'http://www.wikidata.org/entity/Q16971520'), ('lowlbl', 'Woody Allen bibliography'), ('wp', '1950-01-01T00:00:00Z')]
[('aw', 'http://www.wikidata.org/entity/Q13452531'), ('awlbl', 'Commandeur des Arts et des Lettres\u200e'), ('low', 'http://www.wikidata.org/entity/Q16971520'), ('lowlbl', 'Woody Allen bibliography'), ('wp', '1950-01-01T00:00:00Z')]
[('aw', 'http://www.wikidata.org/entity/Q41375'), ('awlbl', 'BAFTA Award for Best Original Screenplay'), ('low', 'http://www.wikidata.org/entity/Q16971520'), ('lowlbl', 'Woody Allen bibliography'), ('wp', '1950-01-01T00:00:00Z')]
[('aw', 'http://www.wikidata.org/entity/Q50616934'), ('awlbl', 'honorary doctorate of Pompeu Fabra University'), ('low', 'http://www.wikidata.org/entity/Q16971520'), ('lowlbl', 'Woody Allen bibliography'), ('wp', '1950-01-01T00:00:00Z')]
[('aw', 'http://www.wikidata.org/entity/Q1111310'), ('awlbl', 'Direct

29

In [61]:
# get node of genders

queryString = """
SELECT DISTINCT ?gender ?glbl
WHERE { 

?film wdt:P31 wd:Q11424 ;
    ?role ?worker .

?film wdt:P57 wd:Q3772 .

?worker wdt:P31 wd:Q5 ;
        wdt:P21 ?gender .
        
?gender <http://schema.org/name> ?glbl .

} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('gender', 'http://www.wikidata.org/entity/Q6581072'), ('glbl', 'female')]
[('gender', 'http://www.wikidata.org/entity/Q6581097'), ('glbl', 'male')]
[('gender', 'http://www.wikidata.org/entity/Q1052281'), ('glbl', 'transgender female')]
[('gender', 'http://www.wikidata.org/entity/Q48270'), ('glbl', 'non-binary')]


4

In [67]:
queryString = """
SELECT ?flbl ?dir (COUNT(DISTINCT ?worker) AS ?num_work) ?fem_worker ?mal_worker
WHERE { 

{
    ?film wdt:P31 wd:Q11424 ;
        ?role ?worker .

    ?film wdt:P57 wd:Q3772 .

    ?worker wdt:P31 wd:Q5 ;
            wdt:P569 ?db .
            
    {
        SELECT ?film (COUNT(DISTINCT ?worker) AS ?fem_worker)
        WHERE
        {
            ?film wdt:P31 wd:Q11424 ;
            ?role ?worker .

            ?film wdt:P57 wd:Q3772 .

            ?worker wdt:P31 wd:Q5 ;
                    wdt:P21 wd:Q6581072 .
        }
        GROUP BY ?film
    }
    
    {
        SELECT ?film (COUNT(DISTINCT ?worker) AS ?mal_worker)
        WHERE
        {
            ?film wdt:P31 wd:Q11424 ;
            ?role ?worker .

            ?film wdt:P57 wd:Q3772 .

            ?worker wdt:P31 wd:Q5 ;
                    wdt:P21 wd:Q6581097 .
        }
        GROUP BY ?film
    }

    ?worker <http://schema.org/name> ?wlbl .
    ?role <http://schema.org/name> ?rlbl .
    ?film <http://schema.org/name> ?flbl .
    wd:Q3772 <http://schema.org/name> ?dir .
}
UNION
{
    ?film wdt:P31 wd:Q11424 ;
        ?role ?worker .

    ?film wdt:P57 wd:Q25089 .

    ?worker wdt:P31 wd:Q5 .
    
    {
        SELECT ?film (COUNT(DISTINCT ?worker) AS ?fem_worker)
        WHERE
        {
            ?film wdt:P31 wd:Q11424 ;
            ?role ?worker .

            ?film wdt:P57 wd:Q25089 .

            ?worker wdt:P31 wd:Q5 ;
                    wdt:P21 wd:Q6581072 .
        }
        GROUP BY ?film
    }
    
    {
        SELECT ?film (COUNT(DISTINCT ?worker) AS ?mal_worker)
        WHERE
        {
            ?film wdt:P31 wd:Q11424 ;
            ?role ?worker .

            ?film wdt:P57 wd:Q25089 .

            ?worker wdt:P31 wd:Q5 ;
                    wdt:P21 wd:Q6581097 .
        }
        GROUP BY ?film
    }

    ?worker <http://schema.org/name> ?wlbl .
    ?role <http://schema.org/name> ?rlbl .
    ?film <http://schema.org/name> ?flbl .
    wd:Q25089 <http://schema.org/name> ?dir .
}

FILTER (?role != wdt:P57)

} 
GROUP BY ?flbl ?dir ?fem_worker ?mal_worker
ORDER BY ?flbl
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('flbl', "A Midsummer Night's Sex Comedy"), ('dir', 'Woody Allen'), ('num_work', '11'), ('fem_worker', '4'), ('mal_worker', '7')]
[('flbl', 'A Rainy Day in New York'), ('dir', 'Woody Allen'), ('num_work', '15'), ('fem_worker', '9'), ('mal_worker', '6')]
[('flbl', 'Alice'), ('dir', 'Woody Allen'), ('num_work', '35'), ('fem_worker', '18'), ('mal_worker', '17')]
[('flbl', 'Annie Hall'), ('dir', 'Woody Allen'), ('num_work', '38'), ('fem_worker', '12'), ('mal_worker', '26')]
[('flbl', 'Another Woman'), ('dir', 'Woody Allen'), ('num_work', '26'), ('fem_worker', '11'), ('mal_worker', '15')]
[('flbl', 'Anything Else'), ('dir', 'Woody Allen'), ('num_work', '13'), ('fem_worker', '6'), ('mal_worker', '7')]
[('flbl', 'Bananas'), ('dir', 'Woody Allen'), ('num_work', '21'), ('fem_worker', '4'), ('mal_worker', '17')]
[('flbl', 'Blue Jasmine'), ('dir', 'Woody Allen'), ('num_work', '21'), ('fem_worker', '5'), ('mal_worker', '16')]
[('flbl', 'Broadway Danny Rose'), ('dir', 'Woody Allen'), ('num

40

In [87]:
# get average age for each film of Woody Allen
# Note: it is a test about the datatime datatype (birth date)

queryString = """
SELECT ?film (ROUND(AVG(?age)) AS ?avg_age)
WHERE { 

?film wdt:P31 wd:Q11424 ;
    ?role ?worker .

?film wdt:P57 wd:Q3772 .

?worker wdt:P31 wd:Q5 ;
        wdt:P569 ?db .
            
BIND(YEAR(NOW()) - YEAR(?db) AS ?age)

} 
GROUP BY ?film
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('avg_year', '61')]
[('film', 'http://www.wikidata.org/entity/Q104123'), ('avg_year', '65')]
[('film', 'http://www.wikidata.org/entity/Q571032'), ('avg_year', '62')]
[('film', 'http://www.wikidata.org/entity/Q1137310'), ('avg_year', '55')]
[('film', 'http://www.wikidata.org/entity/Q72962'), ('avg_year', '69')]
[('film', 'http://www.wikidata.org/entity/Q928019'), ('avg_year', '62')]
[('film', 'http://www.wikidata.org/entity/Q47300912'), ('avg_year', '61')]
[('film', 'http://www.wikidata.org/entity/Q431252'), ('avg_year', '70')]
[('film', 'http://www.wikidata.org/entity/Q165325'), ('avg_year', '60')]
[('film', 'http://www.wikidata.org/entity/Q192115'), ('avg_year', '54')]
[('film', 'http://www.wikidata.org/entity/Q153723'), ('avg_year', '62')]
[('film', 'http://www.wikidata.org/entity/Q18126825'), ('avg_year', '61')]


12

In [99]:
# get average age for each film of Woody Allen
# Note: it is a test about the datatime datatype (work of period)

queryString = """
SELECT ?film (ROUND(AVG(?year)) AS ?avg_year) (datatype(?wp) AS ?type)
WHERE { 

?film wdt:P31 wd:Q11424 ;
    ?role ?worker .

?film wdt:P57 wd:Q3772 .

?worker wdt:P31 wd:Q5 ;
        wdt:P2031 ?wp .
            
BIND(YEAR(?wp) AS ?year)

} 
GROUP BY ?film ?wp
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q571032'), ('avg_year', '2010'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('avg_year', '1981'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q47300912'), ('avg_year', '1968'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q153723'), ('avg_year', '2005'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q153723'), ('avg_year', '1996'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q431252'), ('avg_year', '1979'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q47300912'), ('avg_year', '1956'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q47300912'), ('avg_year', '1950'),

249

In [104]:
# get publication year for each film of Woody Allen

queryString = """
SELECT ?film (YEAR(?pub_year) AS ?pub_y) (datatype(?pub_year) AS ?type)
WHERE { 

?film wdt:P31 wd:Q11424 .

?film wdt:P57 wd:Q3772 .
    
OPTIONAL { ?film wdt:P577 ?pub_year }

} 
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('pub_y', '2015'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('pub_y', '2016'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('pub_y', '2016'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q104123'), ('pub_y', '1994'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q104123'), ('pub_y', '1994'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q104123'), ('pub_y', '1994'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q1137310'), ('pub_y', '2007'), ('type', 'http://www.w3.org/2001/XMLSchema#dateTime')]
[('film', 'http://www.wikidata.org/entity/Q1137310'), ('pub_y', '2007'), ('type', 'http://www.w3

28

In [108]:
# GOAL (4)
# get information about each film of Woody Allen and Quentin Tarantino
# 
# Data:
# - film name
# - director
# - number of workers
# - female workers
# - male workers
# - age average
# - work period start


queryString = """
SELECT ?flbl ?dir (COUNT(DISTINCT ?worker) AS ?num_work) ?fem_worker ?mal_worker (ROUND(AVG(?age)) AS ?avg_age) (ROUND(AVG(?wp_year)) AS ?avg_work_period)
WHERE { 

{
    ?film wdt:P31 wd:Q11424 ;
        ?role ?worker .

    ?film wdt:P57 wd:Q3772 ;
        wdt:P577 ?pub_year .

    ?worker wdt:P31 wd:Q5 ;
            wdt:P569 ?db ;
            wdt:P2031 ?wp .
    {
        SELECT ?film (COUNT(DISTINCT ?worker) AS ?fem_worker)
        WHERE
        {
            ?film wdt:P31 wd:Q11424 ;
            ?role ?worker .

            ?film wdt:P57 wd:Q3772 .

            ?worker wdt:P31 wd:Q5 ;
                    wdt:P21 wd:Q6581072 .
        }
        GROUP BY ?film
    }
    
    {
        SELECT ?film (COUNT(DISTINCT ?worker) AS ?mal_worker)
        WHERE
        {
            ?film wdt:P31 wd:Q11424 ;
            ?role ?worker .

            ?film wdt:P57 wd:Q3772 .

            ?worker wdt:P31 wd:Q5 ;
                    wdt:P21 wd:Q6581097 .
        }
        GROUP BY ?film
    }

    ?worker <http://schema.org/name> ?wlbl .
    ?role <http://schema.org/name> ?rlbl .
    ?film <http://schema.org/name> ?flbl .
    wd:Q3772 <http://schema.org/name> ?dir .
    
    BIND(YEAR(?pub_year) - YEAR(?db) AS ?age) 
    BIND(YEAR(?wp) AS ?wp_year)
    
}
UNION
{
    ?film wdt:P31 wd:Q11424 ;
        ?role ?worker .

    ?film wdt:P57 wd:Q25089 ;
        wdt:P577 ?pub_year .

    ?worker wdt:P31 wd:Q5 ;
            wdt:P569 ?db ;
            wdt:P2031 ?wp .
            
    
    {
        SELECT ?film (COUNT(DISTINCT ?worker) AS ?fem_worker)
        WHERE
        {
            ?film wdt:P31 wd:Q11424 ;
            ?role ?worker .

            ?film wdt:P57 wd:Q25089 .

            ?worker wdt:P31 wd:Q5 ;
                    wdt:P21 wd:Q6581072 .
        }
        GROUP BY ?film
    }
    
    {
        SELECT ?film (COUNT(DISTINCT ?worker) AS ?mal_worker)
        WHERE
        {
            ?film wdt:P31 wd:Q11424 ;
            ?role ?worker .

            ?film wdt:P57 wd:Q25089 .

            ?worker wdt:P31 wd:Q5 ;
                    wdt:P21 wd:Q6581097 .
        }
        GROUP BY ?film
    }

    ?worker <http://schema.org/name> ?wlbl .
    ?role <http://schema.org/name> ?rlbl .
    ?film <http://schema.org/name> ?flbl .
    wd:Q25089 <http://schema.org/name> ?dir .
    
    
    BIND(YEAR(?pub_year) - YEAR(?db) AS ?age) 
    BIND(YEAR(?wp) AS ?wp_year)
    
}

FILTER (?role != wdt:P57)

} 
GROUP BY ?flbl ?dir ?fem_worker ?mal_worker
ORDER BY ?flbl
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('flbl', "A Midsummer Night's Sex Comedy"), ('dir', 'Woody Allen'), ('num_work', '10'), ('fem_worker', '4'), ('mal_worker', '7'), ('avg_age', '56'), ('avg_work_period', '1948')]
[('flbl', 'A Rainy Day in New York'), ('dir', 'Woody Allen'), ('num_work', '13'), ('fem_worker', '9'), ('mal_worker', '6'), ('avg_age', '47'), ('avg_work_period', '1990')]
[('flbl', 'Alice'), ('dir', 'Woody Allen'), ('num_work', '29'), ('fem_worker', '18'), ('mal_worker', '17'), ('avg_age', '47'), ('avg_work_period', '1968')]
[('flbl', 'Annie Hall'), ('dir', 'Woody Allen'), ('num_work', '25'), ('fem_worker', '12'), ('mal_worker', '26'), ('avg_age', '40'), ('avg_work_period', '1961')]
[('flbl', 'Another Woman'), ('dir', 'Woody Allen'), ('num_work', '21'), ('fem_worker', '11'), ('mal_worker', '15'), ('avg_age', '47'), ('avg_work_period', '1965')]
[('flbl', 'Anything Else'), ('dir', 'Woody Allen'), ('num_work', '12'), ('fem_worker', '6'), ('mal_worker', '7'), ('avg_age', '46'), ('avg_work_period', '1980')

50

## Task 5

In [14]:
# GOAL (5)
# get comparison between Woody Allen and Quentin Tarantino
# 
# Data:
# - director
# - birth date
# - work period start
# - number of films
# - number of awards
# - number of nominations
# - number of shared workers
# - name of most frequent worker
# - frequence

queryString = """
SELECT DISTINCT ?dir (?db AS ?birth_date) (?wp_year AS ?start_work) (COUNT(DISTINCT ?film) AS ?num_films) (COUNT(DISTINCT ?aw) AS ?awards) (COUNT(DISTINCT ?nom_for) AS ?nominations) (COUNT(DISTINCT ?worker) AS ?shared_w)
                ?fwlbl ?freq
WHERE { 

{
    wd:Q3772 wdt:P569 ?db ;
            wdt:P2031 ?wp ;
            wdt:P1411 ?nom_for ;
            wdt:P166 ?aw .
    
    ?film wdt:P31 wd:Q11424 ;
        ?role ?worker .

    ?film wdt:P57 wd:Q3772 .

    ?worker wdt:P31 wd:Q5 .
            
    
    ?role <http://schema.org/name> ?rlbl .
    ?film <http://schema.org/name> ?flbl .
    wd:Q3772 <http://schema.org/name> ?dir .
    
    BIND(YEAR(?wp) AS ?wp_year)
    
    {
        SELECT DISTINCT ?worker
        WHERE {
        
            ?film wdt:P31 wd:Q11424 ;
                ?role ?worker .

            ?film wdt:P57 wd:Q25089 .

            ?worker wdt:P31 wd:Q5 .
            
            FILTER (?role != wdt:P57)
        
        }
    }
    
    {
        SELECT (?worker AS ?freq_worker) ?fwlbl (COUNT(?worker) AS ?freq)
        WHERE {
        
            ?film wdt:P31 wd:Q11424 ;
                ?role ?worker .

            ?film wdt:P57 wd:Q3772 .

            ?worker wdt:P31 wd:Q5 .
        
            ?worker <http://schema.org/name> ?fwlbl .
            
            FILTER (?worker != wd:Q3772)
        }
        GROUP BY ?worker ?fwlbl
        ORDER BY DESC (?freq)
        LIMIT 1
    }
}
UNION
{
    wd:Q25089 wdt:P569 ?db ;
            wdt:P2031 ?wp ;
            wdt:P1411 ?nom_for ;
            wdt:P166 ?aw .
    
    ?film wdt:P31 wd:Q11424 ;
        ?role ?worker .

    ?film wdt:P57 wd:Q25089 .

    ?worker wdt:P31 wd:Q5 .
            
    ?role <http://schema.org/name> ?rlbl .
    ?film <http://schema.org/name> ?flbl .
    wd:Q25089 <http://schema.org/name> ?dir .
     
    BIND(YEAR(?wp) AS ?wp_year)
    
    {
        SELECT DISTINCT ?worker
        WHERE {
        
            ?film wdt:P31 wd:Q11424 ;
                ?role ?worker .

            ?film wdt:P57 wd:Q3772 .

            ?worker wdt:P31 wd:Q5 .
        
        }
    }
    
    {
        SELECT (?worker AS ?freq_worker) ?fwlbl (COUNT(?worker) AS ?freq)
        WHERE {
        
            ?film wdt:P31 wd:Q11424 ;
                ?role ?worker .

            ?film wdt:P57 wd:Q25089 .

            ?worker wdt:P31 wd:Q5 .
        
            ?worker <http://schema.org/name> ?fwlbl .
            
            FILTER (?worker != wd:Q25089)
        }
        GROUP BY ?worker ?fwlbl
        ORDER BY DESC (?freq)
        LIMIT 1
    }
    
}

FILTER (?role != wdt:P57)

} 
GROUP BY ?dir ?db ?wp_year ?fwlbl ?freq
"""

print("Results")
run_query(queryString)

Results
[('dir', 'Quentin Tarantino'), ('birth_date', '1963-03-27T00:00:00Z'), ('start_work', '1983'), ('num_films', '11'), ('awards', '16'), ('nominations', '4'), ('shared_w', '16'), ('fwlbl', 'Lawrence Bender'), ('freq', '10')]
[('dir', 'Woody Allen'), ('birth_date', '1935-12-01T00:00:00Z'), ('start_work', '1950'), ('num_films', '14'), ('awards', '32'), ('nominations', '4'), ('shared_w', '16'), ('fwlbl', 'Letty Aronson'), ('freq', '22')]


2

## Task 6

In [6]:
# get predicates from Woody Allen's films

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 

?film wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    ?p ?o .
    
?p <http://schema.org/name> ?plbl .

FILTER (!CONTAINS (?plbl, "ID") && !CONTAINS (?plbl, "username") && !CONTAINS (?plbl, "code"))

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8895'), ('plbl', 'All the Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P2629'), ('plbl', 'BBFC rating')]
[('p', 'http://www.wikidata.org/prop/direct/P2758'), ('plbl', 'CNC film rating (France)')]
[('p', 'http://www.wikidata.org/prop/direct/P3402'), ('plbl', 'CNC film rating (Romania)')]
[('p', 'http://www.wikidata.org/prop/direct/P4276'), ('plbl', 'Cinémathèque québécoise work identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P3216'), ('plbl', 'ClassInd rating')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('plbl', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P2756'), ('plbl', 'EIRIN film rating')]
[('p', 'http://www.wikidata.org/prop/direct/P4437'), ('plbl', 'FPB rating')]
[('p', 'http://www.wikidata.org/prop/direct/P1981'), ('plbl', 'FSK film rating')]
[('p', 'http://www.wikidata.org/prop/direct/P2747'), ('plbl', 'Filmiroda rating')]
[('p', 'http://www.wikidata.org/prop/direct

82

In [7]:
# get assessments from Quentin Tarantino's films

queryString = """
SELECT DISTINCT ?ass ?albl
WHERE { 

?film wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P5021 ?ass .
    
?ass <http://schema.org/name> ?albl .

} 
"""

print("Results")
run_query(queryString)

Results
[('ass', 'http://www.wikidata.org/entity/Q102180036'), ('albl', 'Riz Test')]
[('ass', 'http://www.wikidata.org/entity/Q104144087'), ('albl', 'Waithe Test')]
[('ass', 'http://www.wikidata.org/entity/Q104144383'), ('albl', 'Shukla Test')]
[('ass', 'http://www.wikidata.org/entity/Q104144557'), ('albl', 'Feldman Test')]
[('ass', 'http://www.wikidata.org/entity/Q104144947'), ('albl', 'Landau Test')]
[('ass', 'http://www.wikidata.org/entity/Q105776216'), ('albl', 'reverse Bechdel Test')]
[('ass', 'http://www.wikidata.org/entity/Q4165246'), ('albl', 'Bechdel test')]
[('ass', 'http://www.wikidata.org/entity/Q85783379'), ('albl', 'Mako Mori test')]
[('ass', 'http://www.wikidata.org/entity/Q93605402'), ('albl', 'Vito Russo test')]
[('ass', 'http://www.wikidata.org/entity/Q105640076'), ('albl', 'Topside Test')]
[('ass', 'http://www.wikidata.org/entity/Q105641628'), ('albl', 'Coates Test')]
[('ass', 'http://www.wikidata.org/entity/Q105700813'), ('albl', 'Russ Test')]
[('ass', 'http://www.w

20

In [8]:
# get costs from Quentin Tarantino's films

queryString = """
SELECT DISTINCT ?film ?cost
WHERE { 

?film wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P2130 ?cost .
} 
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18225084'), ('cost', '44000000')]
[('film', 'http://www.wikidata.org/entity/Q104123'), ('cost', '8000000')]
[('film', 'http://www.wikidata.org/entity/Q192115'), ('cost', '40000000')]
[('film', 'http://www.wikidata.org/entity/Q165325'), ('cost', '30000000')]
[('film', 'http://www.wikidata.org/entity/Q431252'), ('cost', '12000000')]
[('film', 'http://www.wikidata.org/entity/Q153723'), ('cost', '70000000')]
[('film', 'http://www.wikidata.org/entity/Q928019'), ('cost', '30000000')]
[('film', 'http://www.wikidata.org/entity/Q571032'), ('cost', '100000000')]


8

In [13]:
# GOAL (6)
# get the films of each director with highter budget
# ANSWER: YES

queryString = """
SELECT DISTINCT ?dir ?flbl ?cost
WHERE { 

{
    {
        SELECT ?dir ?film ?cost
        WHERE {
            ?film wdt:P31 wd:Q11424 ;
                wdt:P57 wd:Q3772 ;
                wdt:P2130 ?cost .                
                
            wd:Q3772 <http://schema.org/name> ?dir .
        }
        ORDER BY DESC (?cost)
        LIMIT 1
    }
    
    ?film <http://schema.org/name> ?flbl .
}
UNION
{        
    {
        SELECT ?dir ?film ?cost
        WHERE {
            ?film wdt:P31 wd:Q11424 ;
                wdt:P57 wd:Q25089 ;
                wdt:P2130 ?cost .
                
            wd:Q25089 <http://schema.org/name> ?dir .
        }
        ORDER BY DESC (?cost)
        LIMIT 1
    }
    
    ?film <http://schema.org/name> ?flbl .
}

} 
"""

print("Results")
run_query(queryString)

Results
[('dir', 'Quentin Tarantino'), ('flbl', 'Django Unchained'), ('cost', '100000000')]
[('dir', 'Woody Allen'), ('flbl', 'You Will Meet a Tall Dark Stranger'), ('cost', '22000000')]


2

## Task 7.3

In [15]:
# GOAL (7.3) PART 1
# get award for Quentin Tarantino's films

queryString = """
SELECT DISTINCT ?aw ?awlbl
WHERE { 

?film wdt:P31 wd:Q11424 ;
    wdt:P57 wd:Q3772 ;
    wdt:P166 ?aw .
    
?aw <http://schema.org/name> ?awlbl .

} 
ORDER BY ?awlbl
"""

print("Results")
run_query(queryString)

Results
[('aw', 'http://www.wikidata.org/entity/Q277751'), ('awlbl', 'Academy Award for Best Production Design')]
[('aw', 'http://www.wikidata.org/entity/Q488651'), ('awlbl', 'Academy Award for Best Original Score')]
[('aw', 'http://www.wikidata.org/entity/Q41417'), ('awlbl', 'Academy Award for Best Writing, Original Screenplay')]
[('aw', 'http://www.wikidata.org/entity/Q106291'), ('awlbl', 'Academy Award for Best Supporting Actor')]
[('aw', 'http://www.wikidata.org/entity/Q41375'), ('awlbl', 'BAFTA Award for Best Original Screenplay')]
[('aw', 'http://www.wikidata.org/entity/Q548389'), ('awlbl', 'BAFTA Award for Best Actor in a Supporting Role')]
[('aw', 'http://www.wikidata.org/entity/Q3404523'), ('awlbl', 'David di Donatello for Best Foreign Actor')]
[('aw', 'http://www.wikidata.org/entity/Q849124'), ('awlbl', 'Golden Globe Award for Best Screenplay')]
[('aw', 'http://www.wikidata.org/entity/Q1170507'), ('awlbl', 'Independent Spirit Award for Best Screenplay')]
[('aw', 'http://www.w

27

## Task 7.1

In [17]:
# get node of which "Academy Award for Best Production Design" is instance 

queryString = """
SELECT DISTINCT ?o ?olbl
WHERE { 

wd:Q277751 wdt:P31 ?o .
    
?o <http://schema.org/name> ?olbl .

} 
ORDER BY ?olbl
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q19020'), ('olbl', 'Academy Awards')]
[('o', 'http://www.wikidata.org/entity/Q96474698'), ('olbl', 'award for best production design')]


2

In [18]:
# GOAL (7.1)
# get all the type of Academy's awards

queryString = """
SELECT DISTINCT ?aw ?awlbl
WHERE { 

?aw wdt:P31 wd:Q19020 .
    
?aw <http://schema.org/name> ?awlbl .

} 
ORDER BY ?awlbl
"""

print("Results")
run_query(queryString)

Results
[('aw', 'http://www.wikidata.org/entity/Q112243'), ('awlbl', 'Academy Award for Best Original Song')]
[('aw', 'http://www.wikidata.org/entity/Q3886464'), ('awlbl', 'Academy Award for Best Live Action Short Film, Comedy')]
[('aw', 'http://www.wikidata.org/entity/Q22117120'), ('awlbl', 'Academy Award for Best Writing')]
[('aw', 'http://www.wikidata.org/entity/Q103916'), ('awlbl', 'Academy Award for Best Actor')]
[('aw', 'http://www.wikidata.org/entity/Q102427'), ('awlbl', 'Academy Award for Best Picture')]
[('aw', 'http://www.wikidata.org/entity/Q107258'), ('awlbl', 'Academy Award for Best Writing, Adapted Screenplay')]
[('aw', 'http://www.wikidata.org/entity/Q498690'), ('awlbl', 'Academy Award for Best Dance Direction')]
[('aw', 'http://www.wikidata.org/entity/Q1425151'), ('awlbl', 'Academy Award for Best Documentary (Short Subject)')]
[('aw', 'http://www.wikidata.org/entity/Q1401974'), ('awlbl', 'Academy Award for Best Short Subject, Two-reel')]
[('aw', 'http://www.wikidata.org

63

## Task 7.2

In [19]:
# GOAL (7.2) PART 1
# get predicates from "Academy Awards"

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 

wd:Q19020 ?p ?o .
    
?p <http://schema.org/name> ?plbl .

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9084'), ('plbl', 'ABC News topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9629'), ('plbl', 'Armeniapedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8295'), ('plbl', 'AustLit ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6200'), ('plbl', 'BBC News topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('plbl', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('plbl', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5019'), ('plbl', 'Brockhaus Enzyklopädie online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5905'), ('plbl', 'Comic Vine ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('plbl', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('plbl', 'Commons gallery')]
[('p', 'http://www.wikidata.org/prop/direct/P3569'), ('plbl', 'Cultureel Woordenboek ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3509'), 

66

In [20]:
# GOAL (7.2) PART 2
# get predicates from Academy's awards

queryString = """
SELECT DISTINCT ?p ?plbl
WHERE { 

?aw wdt:P31 wd:Q19020 ;
    ?p ?o .
    
?p <http://schema.org/name> ?plbl .

} 
ORDER BY ?plbl
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('plbl', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('plbl', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('plbl', 'Commons gallery')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('plbl', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6262'), ('plbl', 'Fandom article ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('plbl', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7502'), ('plbl', 'Golden ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('plbl', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('plbl', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3553'), ('plbl', 'Zhihu topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('plbl', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P2517'), ('plbl', 'category for recipien

32

In [22]:
# GOAL (7.3) PART 2
# get award for Quentin Tarantino

queryString = """
SELECT DISTINCT ?aw ?awlbl
WHERE { 

wd:Q3772 wdt:P166 ?aw .

?aw wdt:P31 wd:Q19020 .
    
?aw <http://schema.org/name> ?awlbl .

} 
ORDER BY ?awlbl
"""

print("Results")
run_query(queryString)

Results
[('aw', 'http://www.wikidata.org/entity/Q41417'), ('awlbl', 'Academy Award for Best Writing, Original Screenplay')]


1

## Task 7

In [51]:
# get awards and nominations of Quentin Tarantino and his films

queryString = """
SELECT ?dir ?film_awards ?film_nominations ?dir_awards ?dir_nominations
WHERE {

    {
        SELECT ?dir (COUNT(?f_aw) AS ?film_awards)
        WHERE {
            ?film wdt:P31 wd:Q11424 ;
                wdt:P57 wd:Q3772 .

            ?film wdt:P166 ?f_aw .

            ?f_aw wdt:P31 wd:Q19020 .

            wd:Q3772 <http://schema.org/name> ?dir .
        }
        GROUP BY ?dir
    }
    
    {
    
        SELECT ?dir (COUNT(?f_nom) AS ?film_nominations)
        WHERE {
        
            ?film wdt:P31 wd:Q11424 ;
                wdt:P57 wd:Q3772 .

            ?film wdt:P1411 ?f_nom .

            ?f_nom wdt:P31 wd:Q19020 .
            
            wd:Q3772 <http://schema.org/name> ?dir .
        }
        GROUP BY ?dir
    
    }
    
    {
        SELECT ?dir (COUNT(?d_aw) AS ?dir_awards)
        WHERE {
        
            wd:Q3772 wdt:P166 ?d_aw .

            ?d_aw wdt:P31 wd:Q19020 .
            
            wd:Q3772 <http://schema.org/name> ?dir .
        }
        GROUP BY ?dir

    }

    {
        SELECT ?dir (COUNT(?d_nom) AS ?dir_nominations)
        WHERE {
        
            wd:Q3772 wdt:P1411 ?d_nom .

            ?d_nom wdt:P31 wd:Q19020 .
            
            wd:Q3772 <http://schema.org/name> ?dir .
        
        }
        GROUP BY ?dir
    }
}
"""

print("Results")
run_query(queryString)

Results
[('dir', 'Quentin Tarantino'), ('film_awards', '7'), ('film_nominations', '24'), ('dir_awards', '1'), ('dir_nominations', '2')]


1

In [ ]:
# GOAL (7)
# get awards and nominations about both directors and their films

queryString = """
SELECT ?dir ?dir_awards ?dir_nominations ?film_awards ?film_nominations
WHERE 
{
    {
        {
            SELECT ?dir (COUNT(?f_aw) AS ?film_awards)
            WHERE {
                ?film wdt:P31 wd:Q11424 ;
                    wdt:P57 wd:Q3772 .

                ?film wdt:P166 ?f_aw .

                ?f_aw wdt:P31 wd:Q19020 .

                wd:Q3772 <http://schema.org/name> ?dir .
            }
            GROUP BY ?dir
        }

        {

            SELECT ?dir (COUNT(?f_nom) AS ?film_nominations)
            WHERE {

                ?film wdt:P31 wd:Q11424 ;
                    wdt:P57 wd:Q3772 .

                ?film wdt:P1411 ?f_nom .

                ?f_nom wdt:P31 wd:Q19020 .

                wd:Q3772 <http://schema.org/name> ?dir .
            }
            GROUP BY ?dir

        }

        {
            SELECT ?dir (COUNT(?d_aw) AS ?dir_awards)
            WHERE {

                wd:Q3772 wdt:P166 ?d_aw .

                ?d_aw wdt:P31 wd:Q19020 .

                wd:Q3772 <http://schema.org/name> ?dir .
            }
            GROUP BY ?dir

        }

        {
            SELECT ?dir (COUNT(?d_nom) AS ?dir_nominations)
            WHERE {

                wd:Q3772 wdt:P1411 ?d_nom .

                ?d_nom wdt:P31 wd:Q19020 .

                wd:Q3772 <http://schema.org/name> ?dir .

            }
            GROUP BY ?dir
        }
    }

UNION

    {
        {
            SELECT ?dir (COUNT(?f_aw) AS ?film_awards)
            WHERE {
                ?film wdt:P31 wd:Q11424 ;
                    wdt:P57 wd:Q25089 .

                ?film wdt:P166 ?f_aw .

                ?f_aw wdt:P31 wd:Q19020 .

                wd:Q25089 <http://schema.org/name> ?dir .
            }
            GROUP BY ?dir
        }

        {

            SELECT ?dir (COUNT(?f_nom) AS ?film_nominations)
            WHERE {

                ?film wdt:P31 wd:Q11424 ;
                    wdt:P57 wd:Q25089 .

                ?film wdt:P1411 ?f_nom .

                ?f_nom wdt:P31 wd:Q19020 .

                wd:Q25089 <http://schema.org/name> ?dir .
            }
            GROUP BY ?dir

        }

        {
            SELECT ?dir (COUNT(?d_aw) AS ?dir_awards)
            WHERE {

                wd:Q25089 wdt:P166 ?d_aw .

                ?d_aw wdt:P31 wd:Q19020 .

                wd:Q25089 <http://schema.org/name> ?dir .
            }
            GROUP BY ?dir

        }

        {
            SELECT ?dir (COUNT(?d_nom) AS ?dir_nominations)
            WHERE {

                wd:Q25089 wdt:P1411 ?d_nom .

                ?d_nom wdt:P31 wd:Q19020 .

                wd:Q25089 <http://schema.org/name> ?dir .

            }
            GROUP BY ?dir
        }
    }
}
ORDER BY ?dir

"""

print("Results")
run_query(queryString)

## Task 7.4

In [69]:
# GOAL (7.4)
# get awards and nominations about both directors and their films
#
# Note: this is a second version of the previous query

queryString = """
SELECT ?dirlbl (COUNT(?d_aw) AS ?dir_awards) (COUNT(?d_nom) AS ?dir_nominations) (COUNT(?f_aw) AS ?film_awards) (COUNT(?f_nom) AS ?film_nominations)
WHERE 
{
    {
        ?dir wdt:P106 wd:Q2526255 ;
            wdt:P166 ?d_aw .

        ?d_aw wdt:P31 wd:Q19020 .

        ?dir <http://schema.org/name> ?dirlbl .

        FILTER (?dir = wd:Q25089 || ?dir = wd:Q3772)
    }
    UNION
    {
        ?dir wdt:P106 wd:Q2526255 ;
            wdt:P1411 ?d_nom .

        ?d_nom wdt:P31 wd:Q19020 .

        ?dir <http://schema.org/name> ?dirlbl .

        FILTER (?dir = wd:Q25089 || ?dir = wd:Q3772)
    }
    UNION
    {
        ?dir wdt:P106 wd:Q2526255 .

        ?film wdt:P31 wd:Q11424 ;
            wdt:P57 ?dir .

        ?film wdt:P166 ?f_aw .

        ?f_aw wdt:P31 wd:Q19020 .

        ?dir <http://schema.org/name> ?dirlbl .

        FILTER (?dir = wd:Q25089 || ?dir = wd:Q3772)
    }
    UNION
    {
        ?dir wdt:P106 wd:Q2526255 .

        ?film wdt:P31 wd:Q11424 ;
            wdt:P57 ?dir .

        ?film wdt:P1411 ?f_nom .

        ?f_nom wdt:P31 wd:Q19020 .

        ?dir <http://schema.org/name> ?dirlbl .

        FILTER (?dir = wd:Q25089 || ?dir = wd:Q3772)
    }
        
}
ORDER BY ?dirlbl

"""

print("Results")
run_query(queryString)

Results
[('dirlbl', 'Quentin Tarantino'), ('dir_awards', '1'), ('dir_nominations', '2'), ('film_awards', '7'), ('film_nominations', '24')]
[('dirlbl', 'Woody Allen'), ('dir_awards', '2'), ('dir_nominations', '3'), ('film_awards', '12'), ('film_nominations', '52')]


2